In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,accuracy_score

import matplotlib.pyplot as plt

In [3]:
iris=load_iris()

X=iris.data
y=iris.target.reshape(-1,1)

print(X.shape,y.shape)

(150, 4) (150, 1)


In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [6]:
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train,dtype=torch.float32)

X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test,dtype=torch.float32)

In [7]:
class irisDataset(Dataset):
  def __init__(self,X,y):
    self.X=X
    self.y=y

  def __len__(self):
    return len(self.X)

  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [8]:
train_dataset=irisDataset(X_train_tensor,y_train_tensor)
test_dataset=irisDataset(X_test_tensor,y_test_tensor)

train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64)

In [9]:
class RegressionDNN(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
    )

  def forward(self, x):
    return self.net(x)

model = RegressionDNN(input_dim=X_train.shape[1])
print(model)

RegressionDNN(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [10]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [11]:
epochs=50
train_losses=[]

for epoch in range(epochs):
  model.train()
  train_loss=0.0
  epoch_loss=0.0
  for X_batch,y_batch in train_loader:
    optimizer.zero_grad()
    predictions=model(X_batch)
    loss=criterion(predictions,y_batch)
    loss.backward()
    optimizer.step()
    epoch_loss +=loss.item()

  epoch_loss=epoch_loss/len(train_loader)
  avg_loss=epoch_loss/len(train_loader)
  train_losses.append(avg_loss)

  if (epoch+1)%10==0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

Epoch [10/50], Loss: 0.0107
Epoch [20/50], Loss: 0.0074
Epoch [30/50], Loss: 0.0073
Epoch [40/50], Loss: 0.0067
Epoch [50/50], Loss: 0.0063


In [14]:
from sklearn.metrics import accuracy_score

with torch.no_grad():
  y_pred = model(X_test_tensor).numpy()

y_true = y_test_tensor.numpy()

mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

y_pred_rounded = np.round(y_pred)
acc = accuracy_score(y_true, y_pred_rounded)

print(f'MAE: {mae:.2f}')
print(f'MSE: {mse:.2f}')
print(f'r2 score: {r2:.4f}')
print(f'accuracy score: {acc:.2f}')

MAE: 0.16
MSE: 0.05
r2 score: 0.9310
accuracy score: 0.97


In [15]:
torch.save(model.state_dict(), 'iris_regression_model.pth')
print('Model saved as iris_regression_model.pth')

Model saved as iris_regression_model.pth
